In [94]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
import sys
root_path = '/content/gdrive/My Drive/TGS-Salt-Segmentation/'
sys.path.append(root_path)

In [0]:
import numpy as np
from keras.models import load_model
import imgaug as ia
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt

# Define Paths and load data

In [0]:
train_img_path = root_path + 'data/128/all_samples/'
model_name = 'ksamxamx.h5'
model_history_path = root_path + 'models/history/'
model_path = root_path + 'models/' + model_name
x_train_path = train_img_path + 'x_train.npy'
y_train_path = train_img_path + 'y_train.npy'
x_valid_path = train_img_path + 'x_valid.npy'
y_valid_path = train_img_path + 'y_valid.npy'

x_train = np.load(x_train_path)
y_train = np.load(y_train_path)
x_valid = np.load(x_valid_path)
y_valid = np.load(y_valid_path)

In [98]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(3200, 128, 128, 1)
(3200, 128, 128, 1)
(800, 128, 128, 1)
(800, 128, 128, 1)


# Define augmentation sequence

In [0]:
# sometimes = lambda aug: iaa.Sometimes(0.5, aug)
"""
augment 50% of the time images: flip left right || affine || crop and pad

"""
aug_sequence = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.OneOf([
        iaa.Noop(),
        iaa.Affine(rotate=(-10, 10), translate_percent={"x": (-0.25, 0.25)}, mode='symmetric', cval=(0), backend="cv2"),
        iaa.Noop(),
        iaa.CropAndPad(
            percent=(-0.2, 0.2),
            pad_mode="reflect",
            pad_cval=0,
            keep_size=False
        ),
    ])
])



def augment(x_train, y_train):
    """
    x_train and y_train will be single images of size (h, w, 1)
    returns the given images augmented
    """
    seq_det = aug_sequence.to_deterministic()
    
    x_train_aug = seq_det.augment_image(x_train)
    y_train_aug = seq_det.augment_image(y_train)
    
    if y_train_aug.shape != x_train.shape or x_train_aug.shape != x_train.shape:
        x_train_aug = ia.imresize_single_image(x_train_aug, x_train.shape[:2], interpolation="linear")
        y_train_aug = ia.imresize_single_image(y_train_aug, x_train.shape[:2], interpolation="nearest")
    return np.array(x_train_aug), np.array(y_train_aug)


aug shapes (128, 128, 1) (128, 128, 1)


# Define image generator with augmentation

In [0]:
# data generator from given images
def image_generator(x_train, y_train, batch_size=32):
    all_batches_index = np.arange(0, x_train.shape[0])
    out_images = []
    out_masks = []
    
    while True:
        np.random.shuffle(all_batches_index)
        for index in all_batches_index:
            img, mask = augment(x_train[index], y_train[index])
       
            out_images.append(img)
            out_masks.append(mask)
            if len(out_images) >= batch_size:
                yield np.stack(out_images, axis=0), np.stack(out_masks, 0)
#                 yield out_images, out_masks
#                 count_idx = 0
                out_images, out_masks = [], []

In [101]:
max=10
for i, m in image_generator(x_train, y_train):
  if max == 0:
    break
  print(i.shape, m.shape)
  max -=1

(32, 128, 128, 1) (32, 128, 128, 1)
(32, 128, 128, 1) (32, 128, 128, 1)
(32, 128, 128, 1) (32, 128, 128, 1)
(32, 128, 128, 1) (32, 128, 128, 1)
(32, 128, 128, 1) (32, 128, 128, 1)
(32, 128, 128, 1) (32, 128, 128, 1)
(32, 128, 128, 1) (32, 128, 128, 1)
(32, 128, 128, 1) (32, 128, 128, 1)
(32, 128, 128, 1) (32, 128, 128, 1)
(32, 128, 128, 1) (32, 128, 128, 1)


In [104]:
model = load_model(model_path)
train_generator = image_generator(x_train, y_train)
epochs = 100

model_callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=1e-9, verbose=1),
    ModelCheckpoint(model_path + model_name, verbose=1, save_best_only=True, save_weights_only=False, monitor='val_loss'),
    TensorBoard(log_dir=model_log_path+model_name)
]

model_progress = model1.fit_generator(
    train_generator,
    steps_per_epoch=x_train.shape[0]*2//batch_size,
    validation_data=[x_valid, y_valid], 
    epochs=epochs,
    callbacks=model_callbacks,
    verbose=1
)

OSError: ignored

In [105]:
!pwd

/content


In [106]:
cd '/content/gdrive/My Drive/TGS-Salt-Segmentation'

/content/gdrive/My Drive/TGS-Salt-Segmentation


In [107]:
!pwd

/content/gdrive/My Drive/TGS-Salt-Segmentation
